## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [Extract entities from mental-health data](https://aws.amazon.com/marketplace/pp/prodview-a7c5q57n2qyq6).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Explain Clinical Document - Mental Health



- **Model**: [en.explain_doc.clinical_mental_health.pipeline](https://nlp.johnsnowlabs.com/2024/05/06/explain_clinical_doc_mental_health_en.html)
- **Model Description**: A pipeline designed for extracting health-related entities from text, assigning assertion statuses to the extracted entities, and establishing relations between the extracted entities.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [ ]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
model_name = "en-explain-doc-clinical-mental-health-pipeline"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [8]:
docs = [
    '''The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI.  She was prescribed 5000 units of Fragmin  subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.''',
    '''The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.'''
]


sample_text = """The patient is a 38-year-old male presenting with complaints of anxiety and depression over the past 2 months.
He has a history of hypertension, generalized anxiety disorder, major depressive disorder.
The patient disclosed a history of childhood trauma, violence and abuse within his household. Those are the contributing factors to his smoking, alcohol use, and concurrent use of cocaine and marijuana.
His anxiety has escalated with increased restlessness, irritability, muscle tension, and difficulties with concentration. He denies any suicidal ideation. Sleep has been poor and appetite decreased.
Current medications include sertraline 100 mg daily, propranolol 20 mg twice daily, and lisinopril 20 mg daily."""

### JSON

In [9]:
input_json_data = {"text": sample_text}
data = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")

ner_df = pd.DataFrame(data[0]["ner_predictions"])
assertion_df = pd.DataFrame(data[0]["assertion_predictions"])
relation_df = pd.DataFrame(data[0]["relation_predictions"])

In [10]:
ner_df

,ner_chunk,begin,end,ner_label,ner_confidence
0,38-year-old,17,27,Age,0.9997
1,male,29,32,Gender,0.9998
2,anxiety,64,70,Mental_Health,0.9849
3,depression,76,85,Mental_Health,0.9939
4,He,111,112,Gender,1.0
5,hypertension,131,142,Disease_Syndrome_Disorder,0.9998
6,generalized anxiety disorder,145,172,Mental_Health,0.4790667
7,major depressive disorder,175,199,Mental_Health,0.5729333
8,childhood trauma,237,252,Childhood_Event,0.77225
9,violence,255,262,Violence_Or_Abuse,0.813


In [11]:
assertion_df

,ner_chunk,begin,end,ner_label,assertion,assertion_confidence
0,anxiety,64,70,Mental_Health,Present,1.0
1,depression,76,85,Mental_Health,Present,0.9979
2,hypertension,131,142,Disease_Syndrome_Disorder,Present,1.0
3,generalized anxiety disorder,145,172,Mental_Health,Present,1.0
4,major depressive disorder,175,199,Mental_Health,Present,1.0
5,childhood trauma,237,252,Childhood_Event,Past,1.0
6,violence,255,262,Violence_Or_Abuse,Past,1.0
7,abuse,268,272,Violence_Or_Abuse,Past,1.0
8,smoking,338,344,Smoking,Hypothetical,0.767
9,alcohol,347,353,Alcohol,Hypothetical,0.6916


In [12]:
relation_df

,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relations,relation_confidence
0,38-year-old,17,27,Age,anxiety,64,70,Mental_Health,Age-Mental_Health,1.0
1,38-year-old,17,27,Age,depression,76,85,Mental_Health,Age-Mental_Health,1.0
2,male,29,32,Gender,anxiety,64,70,Mental_Health,Gender-Mental_Health,1.0
3,male,29,32,Gender,depression,76,85,Mental_Health,Gender-Mental_Health,1.0
4,He,111,112,Gender,generalized anxiety disorder,145,172,Mental_Health,Gender-Mental_Health,1.0
5,He,111,112,Gender,major depressive disorder,175,199,Mental_Health,Gender-Mental_Health,1.0
6,hypertension,131,142,Disease_Syndrome_Disorder,generalized anxiety disorder,145,172,Mental_Health,Disease_Syndrome_Disorder-Mental_Health,1.0
7,hypertension,131,142,Disease_Syndrome_Disorder,major depressive disorder,175,199,Mental_Health,Disease_Syndrome_Disorder-Mental_Health,1.0
8,childhood trauma,237,252,Childhood_Event,violence,255,262,Violence_Or_Abuse,Childhood_Event-Violence_Or_Abuse,1.0
9,childhood trauma,237,252,Childhood_Event,abuse,268,272,Violence_Or_Abuse,Childhood_Event-Violence_Or_Abuse,1.0


### JSON Lines

In [13]:
def create_jsonl(records):
    if isinstance(records, str):
        records = [records]
    json_records = [{"text": text} for text in records]
    json_lines = "\n".join(json.dumps(record) for record in json_records)
    return json_lines

In [14]:
input_jsonl_data = create_jsonl(sample_text)
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"ner_predictions": [{"ner_chunk": "38-year-old", "begin": 17, "end": 27, "ner_label": "Age", "ner_confidence": "0.9997"}, {"ner_chunk": "male", "begin": 29, "end": 32, "ner_label": "Gender", "ner_confidence": "0.9998"}, {"ner_chunk": "anxiety", "begin": 64, "end": 70, "ner_label": "Mental_Health", "ner_confidence": "0.9849"}, {"ner_chunk": "depression", "begin": 76, "end": 85, "ner_label": "Mental_Health", "ner_confidence": "0.9939"}, {"ner_chunk": "He", "begin": 111, "end": 112, "ner_label": "Gender", "ner_confidence": "1.0"}, {"ner_chunk": "hypertension", "begin": 131, "end": 142, "ner_label": "Disease_Syndrome_Disorder", "ner_confidence": "0.9998"}, {"ner_chunk": "generalized anxiety disorder", "begin": 145, "end": 172, "ner_label": "Mental_Health", "ner_confidence": "0.4790667"}, {"ner_chunk": "major depressive disorder", "begin": 175, "end": 199, "ner_label": "Mental_Health", "ner_confidence": "0.5729333"}, {"ner_chunk": "childhood trauma", "begin": 237, "end": 252, "ner_label": 

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [16]:
validation_json_file_name = "input.json"
validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [17]:
# Create JSON and JSON Lines data
input_jsonl_data = create_jsonl(docs)
input_json_data = json.dumps({"text": docs})

# Upload JSON and JSON Lines data to S3
upload_to_s3(input_json_data, validation_json_file_name)
upload_to_s3(input_jsonl_data, validation_jsonl_file_name)

### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [19]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [20]:
retrieve_json_output_from_s3(validation_json_file_name)

[{'ner_predictions': [{'ner_chunk': '30-year-old',
    'begin': 17,
    'end': 27,
    'ner_label': 'Age',
    'ner_confidence': '0.9996'},
   {'ner_chunk': 'female',
    'begin': 29,
    'end': 34,
    'ner_label': 'Gender',
    'ner_confidence': '0.9999'},
   {'ner_chunk': 'insulin',
    'begin': 59,
    'end': 65,
    'ner_label': 'Drug',
    'ner_confidence': '0.9764'},
   {'ner_chunk': 'diabetes',
    'begin': 77,
    'end': 84,
    'ner_label': 'Disease_Syndrome_Disorder',
    'ner_confidence': '0.6342'},
   {'ner_chunk': 'type 2',
    'begin': 87,
    'end': 92,
    'ner_label': 'Disease_Syndrome_Disorder',
    'ner_confidence': '0.96895003'},
   {'ner_chunk': 'She',
    'begin': 95,
    'end': 97,
    'ner_label': 'Gender',
    'ner_confidence': '1.0'},
   {'ner_chunk': 'Bactrim',
    'begin': 120,
    'end': 126,
    'ner_label': 'Drug',
    'ner_confidence': '0.999'},
   {'ner_chunk': 'for 14 days',
    'begin': 128,
    'end': 138,
    'ner_label': 'Drug_Duration',
    'ner_

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [22]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [23]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name)

{"ner_predictions": [{"ner_chunk": "30-year-old", "begin": 17, "end": 27, "ner_label": "Age", "ner_confidence": "0.9996"}, {"ner_chunk": "female", "begin": 29, "end": 34, "ner_label": "Gender", "ner_confidence": "0.9999"}, {"ner_chunk": "insulin", "begin": 59, "end": 65, "ner_label": "Drug", "ner_confidence": "0.9764"}, {"ner_chunk": "diabetes", "begin": 77, "end": 84, "ner_label": "Disease_Syndrome_Disorder", "ner_confidence": "0.6342"}, {"ner_chunk": "type 2", "begin": 87, "end": 92, "ner_label": "Disease_Syndrome_Disorder", "ner_confidence": "0.96895003"}, {"ner_chunk": "She", "begin": 95, "end": 97, "ner_label": "Gender", "ner_confidence": "1.0"}, {"ner_chunk": "Bactrim", "begin": 120, "end": 126, "ner_label": "Drug", "ner_confidence": "0.999"}, {"ner_chunk": "for 14 days", "begin": 128, "end": 138, "ner_label": "Drug_Duration", "ner_confidence": "0.8503334"}, {"ner_chunk": "UTI", "begin": 144, "end": 146, "ner_label": "Disease_Syndrome_Disorder", "ner_confidence": "0.9903"}, {"ner

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

